# Project 2 - Fictitious E-Commerce Dataset
---
DAT 512 Canisius College <br>
Kelsey Dayer <br>
<br>

### Analysis Question
Perform an analysis of the fictitious e-commerce dataset bigquery-public-data.thelook_ecommerce. This analysis will include an analysis of products, users, and products, and users and will address the following:

1) Evaluation of products, analyzing product sales, and product web views, 
and a justified recommendation on cutting 20% of products from the store, listing the products to remove.
<br>

2) Examination of users, user data, purchase and returns behavior, 
and web behavior. Delivering a categorization of users into meaningful segments 
with an explanation of how that categorization could be used by the business. Each user will need to be assigned a category.
<br>

### Sections
- [Product Analysis](#product-analysis)
- [Products to Remove](#product-removal)
- [User Analysis](#user-analysis)
- [User Categorization](#user-cats)

In [71]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()

In [72]:
project_id = 'unit2project'

<a id='product-analysis'></a>
# Product Analysis


In [73]:
#create a dataframe that is a list of all available products
sql = '''
SELECT
   id as product_id, name
FROM
  `bigquery-public-data.thelook_ecommerce.products`
'''

products = pd.io.gbq.read_gbq(sql,project_id=project_id)
products.head()

,product_id,name
0,27569,2XU Men's Swimmers Compression Long Sleeve Top
1,27445,TYR Sport Men's Square Leg Short Swim Suit
2,27457,TYR Sport Men's Solid Durafast Jammer Swim Suit
3,27466,TYR Sport Men's Swim Short/Resistance Short Sw...
4,27481,TYR Alliance Team Splice Jammer


In [74]:
#how many products are listed in the table
products.shape

(29120, 2)

In [75]:
# how many products would need to be cut to hit the 20% goal
29120 * .2

5824.0

In [87]:
#create a dataframe that is a count of how many of each item was sold without being returned or cancelled
sql = '''
SELECT
   product_id, count(product_id) as saleCount
FROM
  `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN
  `bigquery-public-data.thelook_ecommerce.products` p
ON
  oi.product_id = p.id
WHERE status NOT IN ('Returned','Cancelled')
group by product_id
order by saleCount desc
'''

productSales = pd.io.gbq.read_gbq(sql,project_id=project_id)
productSales.head()

,product_id,saleCount
0,24455,16
1,29007,16
2,19592,15
3,16073,15
4,23662,15


In [88]:
productSales.shape

(28817, 2)

In [89]:
#create a dataframe that is a count of how many of each item was viewed
sql = '''
SELECT
   uri as product_id, count(uri) as viewCount
FROM
  `bigquery-public-data.thelook_ecommerce.events` events
WHERE event_type IN ('product')
group by uri
order by viewCount desc
'''

productViews = pd.io.gbq.read_gbq(sql,project_id=project_id)
productViews.head()

,product_id,viewCount
0,/product/19273,67
1,/product/24455,67
2,/product/24161,65
3,/product/29007,64
4,/product/20548,63


In [90]:
#get the unique product id from the uri
uris = productViews['product_id'].values.tolist()
product_id = []
for i in uris:
  temp = i.split("/product/", 1)
  product_id.append(int(temp[-1]))

In [91]:
productViews['product_id'] = product_id
productViews.head() 

,product_id,viewCount
0,19273,67
1,24455,67
2,24161,65
3,29007,64
4,20548,63


In [92]:
productViews.shape

(29120, 2)

In [93]:
# join the product analysis tables to make the product cut decision
productsMerged = pd.merge(productSales, productViews, how="outer", on="product_id")
productsMerged.head()

,product_id,saleCount,viewCount
0,24455,16,67
1,29007,16,64
2,19592,15,48
3,16073,15,58
4,23662,15,59


In [94]:
productsMerged.shape

(29120, 3)

In [95]:
productsMerged.sort_values(by=['saleCount','viewCount'], na_position = 'first', inplace=True)
productsMerged.head()

,product_id,saleCount,viewCount
29119,9853,<NA>,5
29116,3734,<NA>,8
29117,11285,<NA>,8
29118,3954,<NA>,8
29114,407,<NA>,9


<a id='product-removal'></a>
# Products to Remove

In [96]:
productsToRemove = productsMerged.head(5824)
productsToRemove = pd.merge(productsToRemove, products, how="inner", on="product_id")
remove = productsToRemove['name']
remove

0       BedHead Pajamas Women's Long Sleeve Ribbon Paj...
1                         Calvin Klein Women's Cowl Dress
2       Flexees Women's Fat Free Dressing Strapless Ca...
3         Mother of the Bride Formal Evening Dress #27025
4       Patty Women Multi-Style On/Off Shoulder Long S...
                              ...                        
5819    Jones New York Men's 24/7 Total Comfort Sharks...
5820                                 Spidey Kaiten Hoodie
5821         Casual Moments Women's 52 Breakaway Zip Robe
5822    Botany Tall Man Long Sleeve Long Leg Broadclot...
5823    Anna-Kaci S/M Fit Black Sheer Chiffon Dipped T...
Name: name, Length: 5824, dtype: object

I chose the above list of items to remove based on the count of sales and views. I first took the product table and got a count of how many items were available for sale and then I multiplied by .2 to get the number of items that should be cut. I then took the order items table to get the number of sales for each item. This table did not have the same number of rows that the products did so that meant that some products had not had a single sale. I then looked at the event table to get the view count for each product. To do this I took the uri and split off the product id to make analysis using other tables easier. The view table had the same number of rows as the product table. This meant that each product had been viewed at least once. I then merged the sales and views tables to see the products with both of these values. I ordered this table based on sales first and then views. I put the NA values at the top of the table. This left me with the least sold and viewed products at the top. By taking the top 5824 rows and putting them in a new table, I had a table with the bottom 20% of products. I then linked this back with the product table to get the id with the name of the product. 
<br>

I think that ordering by sales and then views to get the least popular items, I was left with the items that could be removed and have the least impact on consumers. It shouldn't push many consumers away and save the company money by not having to stock or advertise the products that generate them the least profit. 

<a id='user-analysis'></a>
# User Analysis

In [103]:
#create a dataframe of the user table
sql = '''
SELECT
   *
FROM
  `bigquery-public-data.thelook_ecommerce.users`
'''

users = pd.io.gbq.read_gbq(sql,project_id=project_id)
users.head()

,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
0,9884,Colleen,Ward,colleenward@example.org,48,F,Mie,249 Joseph Circle,513-0836,Suzuka City,Japan,34.851814,136.508713,Search,2022-08-23 11:40:00+00:00
1,12266,Kathleen,Cruz,kathleencruz@example.net,26,F,Mie,388 Eric Forest Apt. 148,513-0836,Suzuka City,Japan,34.851814,136.508713,Search,2021-02-13 12:16:00+00:00
2,1633,Leroy,Schneider,leroyschneider@example.org,31,M,Acre,9044 Andrade Road,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2020-11-24 17:13:00+00:00
3,26151,Earl,Miller,earlmiller@example.net,30,M,Acre,62069 Rubio Garden,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2019-06-30 14:52:00+00:00
4,31507,Gerald,Wyatt,geraldwyatt@example.org,69,M,Acre,9331 Schwartz Bypass Suite 042,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2020-02-07 00:42:00+00:00


In [104]:
users.shape

(100000, 15)

In [115]:
#create a dataframe that is a count of how many of each item was viewed
sql = '''
SELECT
   user_id as id, uri
FROM
  `bigquery-public-data.thelook_ecommerce.events` events
WHERE event_type IN ('department')
'''

departmentViews = pd.io.gbq.read_gbq(sql,project_id=project_id)
departmentViews.head()

,id,uri
0,93991,/department/women/category/pants&capris/brand/...
1,73117,/department/men/category/underwear/brand/calvi...
2,34174,/department/women/category/intimates/brand/wurl
3,1617,/department/women/category/pants&capris/brand/...
4,93991,/department/women/category/intimates/brand/cor...


In [116]:
departmentViews.shape

(591906, 2)

In [117]:
#get the brand clicks from events
uris = departmentViews['uri'].values.tolist()
brand = []
for i in uris:
  temp = i.split("/brand/", 1)
  brand.append(temp[-1])

In [118]:
departmentViews['uri'] = brand
departmentViews.head() 

,id,uri
0,93991,jonesnewyork
1,73117,calvinklein
2,34174,wurl
3,1617,teezâˆ’her
4,93991,corset-story


In [120]:
#combine user ids and list brand clicks
userBrands = departmentViews.groupby('id')['uri'].apply(list).reset_index(name="brands")

In [121]:
userBrands.head()

,id,brands
0,2,"[vanityfair, myne, myne, thecat'spajamas, thec..."
1,3,[evanpicone]
2,4,[calvinkleinjeans]
3,5,"[goldtoe, plush, plush, goldtoe]"
4,6,"[kiyonna, stellaelyse, hip, hip, calvinkleinje..."


In [122]:
#find most clicked brand for each user id
import statistics
from statistics import mode
 
def most_common(List):
    return(mode(List))
   
brandList = userBrands['brands'].values.tolist()

popBrand = []
for i in brandList:
  temp = most_common(i)
  popBrand.append(temp)

In [123]:
userBrands['brands'] = popBrand
userBrands.head() 

,id,brands
0,2,vanityfair
1,3,evanpicone
2,4,calvinkleinjeans
3,5,goldtoe
4,6,hip


<a id='user-cats'></a>
# User Categorization

In [127]:
#combine user table and the brand, if the user has not clicked on a brand/department they are put in an unkown category
userMerged = pd.merge(userBrands, users, how="outer", on="id")
userMerged.head()

,id,brands,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
0,2,vanityfair,James,Mcmahon,jamesmcmahon@example.org,23,M,Connecticut,37478 Alexander Fields Apt. 154,6484,Shelton,United States,41.305559,-73.137784,Search,2022-01-31 14:16:00+00:00
1,3,evanpicone,Brandy,Espinoza,brandyespinoza@example.org,60,F,Shanghai,486 Pamela Shore,201900,Hangzhou,China,31.361100,121.491316,Search,2021-08-10 07:56:00+00:00
2,4,calvinkleinjeans,Anthony,Hill,anthonyhill@example.net,70,M,Guangdong,738 Nicole Mount,526119,Fuzhou,China,23.050942,112.371023,Email,2020-08-21 14:19:00+00:00
3,5,goldtoe,Lorraine,White,lorrainewhite@example.com,25,F,Jiangxi,69031 Davis Run Suite 266,331100,Matsubara-shi,China,28.192361,115.775801,Facebook,2022-06-04 13:57:00+00:00
4,6,hip,James,Taylor,jamestaylor@example.net,39,M,Beijing,30457 Jose Points,101401,Shanghai,China,40.289544,116.645508,Search,2019-04-08 06:47:00+00:00


In [130]:
userMerged.brands = userMerged.brands.fillna('Unknown')

For user categorization I decided to look at the department clicks of users. First I created a dataframe from the user table and checked to see how many users were listed. I then took department clicks from the events table. I took out the brand substring because I felt that seeing what brands users were shopping would have some information about their shopping habits. I then grouped the brand clicks by user id. I had the user id and then a list of all the brands they have clicked on on the site. I took the list of brand clicks and picked the most common brand from the list to assign to the user. If the user had not clicked on a department I assigned them the category of uknown. The number of users that had clicked on a brand or department was almost 80%. This felt like a high enough percentage that the unknown category would be acceptable. 
<br>

This categorization could be useful to the site for marketing purposes. By knowing the brand that is most shopped by the customer the site can appropriately advertise to them. Pushing brands the user already shows interest in is more likey to result in a click or sale than a brand they avoid or have never heard of. 